<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/answer/13a_struct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第13章 系列データの識別

## 課題1

上記コードはPyStructのサイトで公開されいるサンプルですが、おそらくコーディングミスだと思われる箇所がデータの読み込みから性能比較までの部分にあります。その場所を修正し、正しいと思われるコードで実行してみてください。

In [1]:
# colabで実行
!pip install pystruct
import pkg_resources, imp
imp.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

PyStructのサイトで紹介されている英単語手書き文字系列の認識をやってみます。英小文字をクラスとしているので、単語の最初の文字（大文字）を削除しているという特殊な設定です。

https://pystruct.github.io/auto_examples/plot_letters.html#sphx-glr-auto-examples-plot-letters-py

ライブラリの読み込み

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC

from pystruct.datasets import load_letters
from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM
#from pystruct.learners import StructuredPerceptron

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


正解の表示のために、整数で表された正解データと文字との対応を取るための変数を定義します。

In [3]:
abc = "abcdefghijklmnopqrstuvwxyz"

データの読み込み

各事例は、文字数×128(縦16、横8のバイナリ(0,1)画像をフラットにしたもの）のndarrayです。同じ単語について、異なる人が書いた事例が10事例前後並んでおり、各事例にはデータ分割用の情報fold（0から9の整数値）が付いています。

In [4]:
letters = load_letters()
X, y, folds = letters['data'], letters['labels'], letters['folds']
X, y = np.array(X), np.array(y)
X_train, X_test = X[folds == 1], X[folds != 1]
y_train, y_test = y[folds == 1], y[folds != 1]

事例数を確認します。

In [5]:
print("train: %d  test: %d" % (X_train.shape[0], X_test.shape[0]))

train: 704  test: 6173


学習事例よりも評価事例が圧倒的に多くなっています。trainとtestの振り分けが反対だと思われます。

In [6]:
X_train, X_test = X[folds != 1], X[folds == 1]
y_train, y_test = y[folds != 1], y[folds == 1]
print("train: %d  test: %d" % (X_train.shape[0], X_test.shape[0]))

train: 6173  test: 704


比較用に、各画像単独で scikit-learnのSVCで学習します。

In [7]:
# Train linear SVM
svm = LinearSVC(dual=False, C=.1)
# flatten input
svm.fit(np.vstack(X_train), np.hstack(y_train))

LinearSVC(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

構造化SVMで学習します。（30秒ほどかかります）

In [8]:
# Train linear chain CRF
model = ChainCRF()
ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=11)
ssvm.fit(X_train, y_train)

FrankWolfeSSVM(C=0.1, batch_mode=False, check_dual_every=10, do_averaging=True,
               line_search=True, logger=None, max_iter=11,
               model=ChainCRF(n_states: 26, inference_method: max-product),
               n_jobs=1, random_state=None, sample_method='perm',
               show_loss_every=0, tol=0.001, verbose=0)

構造化の有無で性能を比較します。やはりオリジナルのコードよりも差が出ました。

In [9]:
print("Test score with chain CRF: %f" % ssvm.score(X_test, y_test))

print("Test score with linear SVM: %f" % svm.score(np.vstack(X_test),
                                                   np.hstack(y_test)))

Test score with chain CRF: 0.850047
Test score with linear SVM: 0.747907


## 課題2　（ある程度Pythonでのコーディング経験がある人向け）

テストデータが系列学習の性能評価において適切なものになっているか確認してください。

系列データの学習においては、学習データの系列と評価データの系列の重複の程度を見ます。あまり多く重複しているようなら、入力との対応よりも出力系列の確からしさで答えられている可能性があります。

In [10]:
count = 0
for y1 in y_test:
  for y2 in y_train:
    if len(y1) == len(y2): # 同じ文字数か？
      words = (y1 == y2)   # 1文字毎に一致するか確認
      if np.all(words == True): # 同じ文字列の場合はカウント
        count = count + 1
        break
print("case %d, count %d" % (y_test.shape[0], count))

case 704, count 704


評価用データの全事例が学習用データに含まれてしまっています。これではあまりいい分割とはいえません。未知系列が一定数含まれる評価用データが望ましいですが、学習データの数があまり多くないので、学習データで系列の正しさの傾向が得られているかどうかの判断は難しいところです。